In [1]:
import numpy as np
import pandas as pd
import mlflow

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN

import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sudhirjoon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sudhirjoon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/opt/anaconda3/lib/python3.11/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
mlflow.set_tracking_uri("http://13.60.79.0:5000")
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

<Experiment: artifact_location='s3://capstone-yt-mlflow-bucket/542391321365143307', creation_time=1735485482149, experiment_id='542391321365143307', last_update_time=1735485482149, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={}>

In [3]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [4]:
# Define TF-IDF vectorization parameters
ngram_range = (1, 2)  # Bigram setting
max_features = 1000  # Set max_features to 1000 for TF-IDF

# Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category']
)

# Vectorization using TF-IDF
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)  # Fit on training data
X_test_vec = vectorizer.transform(X_test)  # Transform test data

# Handle imbalance using ADASYN
adasyn = ADASYN(random_state=42)
X_train_vec, y_train = adasyn.fit_resample(X_train_vec, y_train)

# Function to log model and metrics in MLFlow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model name and experiment type
        mlflow.set_tag("mlflow.runName", f"{model_name}_ADASYN_TFIDF_Bigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log model algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model and evaluate
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log detailed classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model itself
        mlflow.sklearn.log_model(model, f"{model_name}_model")
# Optuna objective function for Random Forest
def objective_rf(trial):
    # Suggest hyperparameters
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    max_depth = trial.suggest_int("max_depth", 3, 50)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", None])

    # Create the model with suggested parameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42
    )

    # Fit the model
    model.fit(X_train_vec, y_train)

    # Calculate accuracy on the test set
    y_pred = model.predict(X_test_vec)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy
# Run Optuna to optimize Random Forest hyperparameters
def run_optuna_rf_experiment():
    # Create an Optuna study
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_rf, n_trials=30)

    # Get the best hyperparameters and retrain the best model
    best_params = study.best_params
    best_model = RandomForestClassifier(
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        min_samples_split=best_params['min_samples_split'],
        min_samples_leaf=best_params['min_samples_leaf'],
        max_features=best_params['max_features'],
        random_state=42
    )

    # Log the best model with MLFlow
    log_mlflow("RandomForest", best_model, X_train_vec, X_test_vec, y_train, y_test)

# Run the Optuna experiment for Random Forest
run_optuna_rf_experiment()


[I 2024-12-30 14:31:46,917] A new study created in memory with name: no-name-97416aa0-73da-428c-9748-3002aeb3c265
[I 2024-12-30 14:31:48,844] Trial 0 finished with value: 0.7263057411700532 and parameters: {'n_estimators': 106, 'max_depth': 29, 'min_samples_split': 9, 'min_samples_leaf': 3, 'max_features': 'log2'}. Best is trial 0 with value: 0.7263057411700532.
[I 2024-12-30 14:31:51,961] Trial 1 finished with value: 0.7179871812355108 and parameters: {'n_estimators': 203, 'max_depth': 24, 'min_samples_split': 5, 'min_samples_leaf': 7, 'max_features': 'log2'}. Best is trial 0 with value: 0.7263057411700532.
[I 2024-12-30 14:32:07,888] Trial 2 finished with value: 0.7178508114005182 and parameters: {'n_estimators': 276, 'max_depth': 35, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.7263057411700532.
[I 2024-12-30 14:32:10,978] Trial 3 finished with value: 0.7425337515341607 and parameters: {'n_estimators': 82, 'max_depth': 48, 'mi

🏃 View run RandomForest_ADASYN_TFIDF_Bigrams at: http://13.60.79.0:5000/#/experiments/542391321365143307/runs/e4ea801ee1434355b164a8272ef6de56
🧪 View experiment at: http://13.60.79.0:5000/#/experiments/542391321365143307
